# Characterize how well embeddings resolve different features.  
This file assumes that there are standardized features supplied

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.io as sio
sns.set()
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier,  AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
import plotly
import plotly.graph_objs as go

# Configure Plotly to be rendered inline in the notebook.
plotly.offline.init_notebook_mode()

### Load files

In [ ]:
# loading files from allen file system mat directory
# mount_point    = '/Users/fruity/Remote-AI-root/allen/aibs' #Rohan's mount point
# mount_point    = '/allen/aibs' #Corinne's mount point
# data_dir       = mount_point + 'mat/Corinne/autoencoder/'
#embeddings_dir = mount_point+'mat/Corinne/autoencoder/embeddings/'

#Load Corinne's data file:
#dat_df=pd.read_csv(data_dir+'ae_data_50hz_sequential_10_24_2019.csv', sep='#',low_memory=False)

#Load embeddings:
#mat = sio.loadmat(embeddings_dir+'TEST_cv_0_pd_0-2_bs_1000_ld_2_ne_10000_ri_0-summary.mat, \
#                  squeeze_me=True)

In [ ]:
# loading files locally
# sequential data
df=pd.read_csv('data2_first_8_pulses/ae_data_50hz_sequential_10_24_2019UPDATE.csv', sep='#',low_memory=False)
embed2D = sio.loadmat('data2_first_8_pulses/TEST_cv_0_pd_0-2_bs_1000_ld_2_ne_10000_ri_0-summary.mat', squeeze_me=True)
embed3D = sio.loadmat('data2_first_8_pulses/TEST-3D_cv_0_pd_0-2_bs_1000_ld_3_ne_10000_ri_0-summary.mat', squeeze_me=True)

# random data
# This doesnt yet work due to the key stadarization of the mat file and no 3d embedding
# df=pd.read_csv('data1_random/ae_data_09_06_2019UPDATE.csv', sep='#',low_memory=False)
# embed2D = sio.loadmat('data1_random/std_data_run_0_cv_0_ng_500_pd_0-5_bs_1000_ld_2_ne_5000_ri_0-summaryUPDATE.mat', squeeze_me=True)

df.keys()

### Check file standardization

In [ ]:
features = ['pair', 'species', 'pre_class','post_class', 'pre_cre', 'post_cre',
       'pre_layer', 'post_layer', 'pre_ex', 'post_ex', 'stp_induction_50hz']
if not np.all([f in df.columns for f in features]):
    for f in features:
        if f not in df.columns:
            print("missing", f)
    print('attempting to standarize data')
    
    #dealing with pair
    if 'pair' in df.columns:
        pass
    elif 'pair_identifier' in df.columns:
        df.rename(columns = {'pair_identifier':'pair'})
    elif ('expt' in df.columns) & ('pre_cell' in df.columns) & ('post_cell' in df.columns):
        pass
    else:
        raise Exceptation("cannot resolve missing 'pair' feature") 
    
    #dealing with class
    if 'pre_class' not in df.columns:
        if ('species' in df.columns) & ('pre_cre' in df.columns) & ('pre_layer' in df.columns):
            df['pre_class']=df['pre_cre']
            df['pre_class'].loc[df['species'] == 'human'] = 'human'
            df['pre_class'].loc[(df['pre_cre'] == 'unknown') & (df['species'] == 'mouse') & (df['pre_layer'] == '2/3')] = '2/3'
            df['pre_class'].loc[(df['pre_cre'] == 'unknown') & (df['species'] == 'mouse') & (df['pre_layer'] == '4')] = '4'
            df['pre_class'].loc[(df['pre_cre'] == 'unknown') & (df['species'] == 'mouse') & (df['pre_layer'] == '6')] = '6'
            df['pre_class'].loc[(df['pre_cre'] == 'unknown') & (df['species'] == 'mouse') & (df['pre_layer'] == '5')] = '5'
        else:
            raise Exceptation("cannot resolve missing 'pre_class' feature") 
            
    if 'post_class' not in df.columns:
        if ('species' in df.columns) & ('post_cre' in df.columns) & ('post_layer' in df.columns):
            df['post_class']=df['post_cre']
            df['post_class'].loc[df['species'] == 'human'] = 'human'
            df['post_class'].loc[(df['post_cre'] == 'unknown') & (df['species'] == 'mouse') & (df['post_layer'] == '2/3')] = '2/3'
            df['post_class'].loc[(df['post_cre'] == 'unknown') & (df['species'] == 'mouse') & (df['post_layer'] == '4')] = '4'
            df['post_class'].loc[(df['post_cre'] == 'unknown') & (df['species'] == 'mouse') & (df['post_layer'] == '6')] = '6'
            df['post_class'].loc[(df['post_cre'] == 'unknown') & (df['species'] == 'mouse') & (df['post_layer'] == '5')] = '5'
        else:
            raise Exceptation("cannot resolve missing 'post_class' feature") 

# remove actual data because not needed and takes up memory (actually maybe I do want it.)
df.drop(df.columns.difference(features), 1, inplace=True)

### Create pre-post nurmerical catagories for supervised learning tests

In [ ]:
# #Annotation combinations
# df = df.assign(pre_post_class=(df['pre_class'].map(str) + '_' + df['post_class'].map(str)))
# df = df.assign(pre_post_ex=(df['pre_ex'].map(str) + '_' + df['post_ex'].map(str)))

# #Assign numeric id to 'combined_anno'
# df = df.assign(num_pre_post_class=(df['pre_post_class']).astype('category').cat.codes)
# df = df.assign(num_pre_post_ex=(df['pre_post_ex']).astype('category').cat.codes)

### Create columns for the embedded data

In [ ]:
#random embedding needs updates to work here

# Note that you cannot run this cell twice because you will keep adding to dataframe
if not (embed2D['zX'].shape[0] == df.shape[0]):
    raise Exception('the dimensions of the embedding does not match the data')
df = df.join(pd.DataFrame(embed2D['zX'], columns = ['embed2Da', 'embed2Db']))

if 'embed3D' in locals(): #this is currently here because I don't have 3D embedding for random data yet
    if not ((embed2D['zX'].shape[0] == df.shape[0]) & (embed3D['zX'].shape[0] == df.shape[0])):
        raise Exception('the dimensions of the embedding does not match the data')
    df = df.join(pd.DataFrame(embed3D['zX'], columns = ['embed3Da', 'embed3Db', 'embed3Dc' ]))
else:
    print('there is no 3D embedding')
df.keys()

In [ ]:
embed2D

# SUPERVISED LEARNING: HOW WELL CAN PRE-SYNAPTIC EXICITATORY (E & I) CLASS BE DIFFERENTIATED IN THE LATENT SPACE?

### set up pre synaptic excitation DataFrame 

In [ ]:
# get rid of unknown excitation (this is annoying given that I will have to make a df for )
df_pre_ex = df[df['pre_ex'] != 'U']

#Assign numeric catatory for plotting
df_pre_ex = df_pre_ex.assign(num_pre_ex=(df_pre_ex['pre_ex']).astype('category').cat.codes)

# get a map of catagory codes for making legends (need to circle back)
c = df_pre_ex.pre_ex.astype('category')
d = dict(enumerate(c.cat.categories))
print (d)

### function for plotting and accuracy calculation

In [ ]:
def plot_and_classify(df, num_cat_name, embed_name_list):
    ''' 
    plot the data in the pre_defined classification and assess how well those can be differentiated 
    in the latent space.
    
    input
    -----
    df: DataFrame
    num_cat_name: string
        specifies the name of the column with the numerical catagory
    embed_name_list: list of strings
        specifies the column names of the embedded space to be plotted
    '''
    # create embeded variable for ease
    embed = df[embed_name_list].values
    
    # Train a Random Forrest on this to see how well it can be classified
    y = df[num_cat_name].values
    X_train, X_test, y_train, y_test = train_test_split(embed, y, stratify = y) #split up data based on equal number of y output
    RF =  RandomForestClassifier(max_depth=None, min_samples_split=2).fit(X_train, y_train)
    LR = LogisticRegression().fit(X_train, y_train)
    yRF_predict = RF.predict(X_test)
    yLR_predict = LR.predict(X_test)
    print('RF accuracy', RF.score(X_test, y_test))
    print('LR accuracy', LR.score(X_test, y_test))
    print('naive accuracy', sum(y_test == 1)/len(y_test))
    print('Random forrest confusion', metrics.confusion_matrix(y_test, yRF_predict))
    print('Logistic regression confusion', metrics.confusion_matrix(y_test, yLR_predict))
    
    # make plot
    # note: need to circle back to making a legend
    #make 2D plot
    if len(embed_name_list) == 2:
        plt.figure(figsize = (15,15))
        plt.scatter(embed[:,0], embed[:,1], s=1, c=df[num_cat_name].values, cmap='viridis')
        ax = plt.gca()
        ax.set_xlim(-4,4)
        ax.set_ylim(-4,4)
        
    # make 3D plot
    elif len(embed_name_list) == 3:
        # Configure the trace.
        trace = go.Scatter3d(
        #    x=[1, 2, 3],  # <-- Put your data instead
        #    y=[4, 5, 6],  # <-- Put your data instead
        #    z=[7, 8, 9],  # <-- Put your data instead
            x=embed[:,0], 
            y=embed[:,1],
            z=embed[:,2],
        #    s=1,
            mode = 'markers',
            marker = dict(
                size = 5,
                color=df[num_cat_name].values, 
                colorscale='viridis',
                colorbar = dict()),
        )

        # Configure the layout.
        layout = go.Layout(
            margin={'l': 0, 'r': 0, 'b': 0, 't': 0}
        )

        data = [trace]
        plot_figure = go.Figure(data=data, layout=layout)

        # Render the plot.
        plotly.offline.iplot(plot_figure)
    else:
        raise Exception('plotting dimension not defined')

### 2D embedding

In [ ]:
plot_and_classify(df_pre_ex, 'num_pre_ex', ['embed2Da', 'embed2Db'])

### 3D embedding does help a bit

In [ ]:
plot_and_classify(df_pre_ex, 'num_pre_ex', ['embed3Da', 'embed3Db', 'embed3Dc'])

# Look at post synaptic excitation instead

### set up post synaptic excitation DataFrame 

In [ ]:
# get rid of unknown excitation (this is annoying given that I will have to make a df for )
df_post_ex = df[df['post_ex'] != 'U']

#Assign numeric catatory for plotting
df_post_ex = df_post_ex.assign(num_post_ex=(df_post_ex['post_ex']).astype('category').cat.codes)

# get a map of catagory codes for making legends (need to circle back)
c = df_post_ex.pre_ex.astype('category')
d = dict(enumerate(c.cat.categories))
print (d)

### 2D embedding

In [ ]:
plot_and_classify(df_post_ex, 'num_post_ex', ['embed2Da', 'embed2Db'])

### 3D embedding does help a bit

In [ ]:
plot_and_classify(df_post_ex, 'num_post_ex', ['embed3Da', 'embed3Db', 'embed3Dc'])

### Given that post synaptic excitation did classify much above chance it is unlikey that both together would help but let us check and take the opportunity to look at multiple catagories

### set up pre/post synaptic excitation DataFrame 

In [ ]:
# get rid of unknown excitation (this is annoying given that I will have to make a df for )
df_pre_post_ex = df[~((df['pre_ex'] == 'U') | (df['post_ex'] == 'U'))]

#Assign numeric catatory for plotting
df_pre_post_ex = df_pre_post_ex.assign(pre_post_ex=(df_pre_post_ex['pre_ex'].map(str) + '_' + df_pre_post_ex['post_ex'].map(str)))
df_pre_post_ex = df_pre_post_ex.assign(num_pre_post_ex=(df_pre_post_ex['pre_post_ex']).astype('category').cat.codes)

# get a map of catagory codes for making legends (need to circle back)
c = df_pre_post_ex.pre_post_ex.astype('category')
d = dict(enumerate(c.cat.categories))
print (d)

### 2D embedding

In [ ]:
plot_and_classify(df_pre_post_ex, 'num_pre_post_ex', ['embed2Da', 'embed2Db'])

### 3D embedding does help a bit

In [ ]:
plot_and_classify(df_pre_post_ex, 'num_pre_post_ex', ['embed3Da', 'embed3Db', 'embed3Dc'])

# TODO
* accuracy and baseline and for multiple catagories
* how to differentiate which catagories are unhelpful i.e. have a lot of overlap (for example E2I and E2E largely the same?)  Might need to to a heirarchcal classification schema.
    * perhaps quantify how well each individual class can be differentiated by every other individual class and combine non differentiable
* variance of pair versus class
* distribution of average amplitude, stp, cv accross this space
* what are the outliers


# SUPERVISED LEARNING: HOW WELL CAN CLASS BE DIFFERENTIATED IN THE LATENT SPACE? PRE_SYNAPTIC, POST_SYNAPTIC AND PAIR

### First lets evalutate pre synaptic class

# LOOK AT DISTRIBUTION OF PARAMETERS ACCROSS THE SPACE

In [ ]:
df.keys()

In [ ]:
plt.figure(figsize = (15,15))
plt.scatter(df['embed2Da'], df['embed2Db'], s=1, c=df['stp_induction_50hz'].values, cmap='viridis')
plt.colorbar()
ax = plt.gca()
ax.set_xlim(-4,4)
ax.set_ylim(-4,4)
plt.title('STP Induction', fontsize=24)

In [ ]:
# Configure the trace.
trace = go.Scatter3d(
#    x=[1, 2, 3],  # <-- Put your data instead
#    y=[4, 5, 6],  # <-- Put your data instead
#    z=[7, 8, 9],  # <-- Put your data instead
    x=df['embed3Da'], 
    y=df['embed3Db'],
    z=df['embed3Dc'],
#    s=1,
    mode = 'markers',
    marker = dict(
        size = 5,
        color=df['stp_induction_50hz'].values, 
        colorscale ='viridis',
        showscale = True)
)

# Configure the layout.
layout = go.Layout(
    margin={'l': 0, 'r': 0, 'b': 0, 't': 0}
)

data = [trace]
plot_figure = go.Figure(data=data, layout=layout)

# Render the plot.
plotly.offline.iplot(plot_figure)

# Lets characterize variance of single pairs in the space.  Can we use the mean as a good descriptor of one pair?

### Make a new dataFrame where information is for one pair.

In [ ]:
df_mean=df.groupby(features).mean()
df_var=df.groupby(features).var()
df_stat = pd.concat([df_mean, df_var],  axis=1)

# -----------------BELOW IS DANGEROUS---------------
# There used to be a way to specify subset in the concat but I can't find it therefore I am renaming the columns
df_stat.columns = ['embed2Da_mean', 'embed2Db_mean', 'embed3Da_mean', 'embed3Db_mean',
       'embed3Dc_mean', 'embed2Da_var', 'embed2Db_var', 'embed3Da_var',
       'embed3Db_var', 'embed3Dc_var']

# the multi-index does not appear to be working or be accessable as described.  I tried many different things.  
# I couldnt even convert it to a numpy array and get the data out with indexing one would expect. So I am
# hacking the info out via for loops
anno_dict={}
for name in df_stat.index.names:
    anno_dict[name] = []

for ii in range(len(df_stat.index)):
    for n, name in enumerate(df_stat.index.names):
        anno_dict[name].append(df_stat.index[ii][n])

for key in anno_dict:
    df_stat[key] = anno_dict[key]

# remove the multi-index
df_stat.reset_index(drop=True, inplace = True)

# reorder columns for convenience
cols = df_stat.columns.tolist()
cols = cols[-len(features):] + cols[:-len(features)]
df_stat = df_stat[cols]
df_stat

In [ ]:
plt.errorbar(df_stat['embed2Da_mean'], 
             df_stat['embed2Db_mean'], xerr = df_stat['embed2Da_var'].values), 
   #          c=df_stat['stp_ind_50hz_mean'].values)
#, cmap='viridis')


In [ ]:
plt.figure(figsize = (15,15))
plt.scatter(df_stat['embed2Da_mean'], 
             df_stat['embed2Db_mean'],  
             c=df_stat['embed2Da_var'].values,
             cmap='viridis')
plt.colorbar()
ax = plt.gca()
ax.set_xlim(-4,4)
ax.set_ylim(-4,4)
plt.title('x_variance', fontsize=24)

In [ ]:
# get rid of unknown excitation (this is annoying given that I will have to make a df for )
df_pre_ex = df_stat[df_stat['pre_ex'] != 'U']

#Assign numeric catatory for plotting
df_pre_ex = df_pre_ex.assign(num_pre_ex=(df_pre_ex['pre_ex']).astype('category').cat.codes)

# get a map of catagory codes for making legends (need to circle back)
c = df_pre_ex.pre_ex.astype('category')
d = dict(enumerate(c.cat.categories))
print (d)

In [ ]:
df_pre_ex['pre_ex']